### Label data

In [2]:
from utils.models import DataItem
lines =open('/home/ec2-user/SageMaker/mariano/notebooks/05. High Recall V2/labeled_data.csv','r').read().splitlines()[1:]
lines = [line.split(';') for line in lines ]
lines[:3]
labeled_data = [DataItem(id_) for id_,_ in lines]

idx = 0
for id_,label in lines:
        item = labeled_data[idx]
        idx+=1
        assert item.id_ == id_
        if label == 'R':
            item.set_relevant()
        else:
            item.set_irrelevant()
            assert label=='I'


lines = open('./auxiliary_notebooks/in_canada.csv', 'r').read().splitlines()
irrelevants = [DataItem(line.split(';')[0]) for line in lines if line.split(';')[1]=='False']
for item in irrelevants:
    item.set_irrelevant()
labeled_data = labeled_data + irrelevants
print(f'{len(labeled_data):,}')

ids = set([item.id_ for item in labeled_data])
print(len(ids))

114,492
114492


In [4]:
from utils.general import info,ok,warning
import os

predictions_dirpath = '../04. Model of DP/predictions/'

files = os.listdir(predictions_dirpath)
total_predictions = len([file_ for file_ in files if file_.endswith('_v3.p')])

info(f"Total of files with predictions:                     {total_predictions:,}")
files = [file_ for file_ in files if file_.endswith('_v3.p') if not file_[:-5] in ids]
info(f"Number of prediction ignoring already labeled data = {len(files):,}  (-{total_predictions-len(files):,})")

2022-03-26 20:18:43.303836 [ INFO  ] Total of files with predictions:                     4,973
2022-03-26 20:18:43.305827 [ INFO  ] Number of prediction ignoring already labeled data = 4,906  (-67)


In [12]:
import pandas as pd
import pickle
import numpy as np
from utils.tdmstudio import TDMStudio
from utils.general import id2file
data = {}
data['ID']=[]
data['yhat1']=[]
data['yhat2']=[]
data['yhat3']=[]
data['average']=[]
# data['URL']=[]
data['title']=[]
for file_ in files:
    yhat1,yhat2,yhat3 = pickle.load(open(predictions_dirpath+file_,'rb'))
    if yhat1>0.5 and yhat2>0.5 and yhat3>0.5:
        data['ID'].append(file_[:-5])
        data['yhat1'].append(yhat1)
        data['yhat2'].append(yhat2)
        data['yhat3'].append(yhat3)
        data['average'].append(np.average([yhat1,yhat2,yhat3]))
        data['title'].append(TDMStudio.get_title(id2file(file_[:-5])))
#         data['URL'].append(f'https://proquest.com/docview/{file_[:-5]}')
    
#     data.append((file_,pickle.load(open(predictions_dirpath+file_,'rb'))))

info(f"Number of irrelevant (<0.5) articles discarded: {len(files)-len(data['ID']):,} ({len(files)}-{len(data['ID'])})")
df = pd.DataFrame(data).sort_values(by=['average'],ascending=False)
df

2022-03-26 20:24:34.070768 [ INFO  ] Number of irrelevant (<0.5) articles discarded: 4,444 (4906-462)


,ID,yhat1,yhat2,yhat3,average,title
156,1287517171,0.999998,0.999999,1.000000,0.999999,ADMIT MORE DP's
332,1291317371,0.996367,1.000000,1.000000,0.998789,More DP's Arrive
274,1313779779,0.995022,0.999999,0.999985,0.998335,PARTY ARRANGED FOR 250 DP GIRLS
360,1291252277,1.000000,0.994635,1.000000,0.998212,78 DP Girls Reach Ontario To Take Position As ...
58,1291513884,0.987245,0.996306,1.000000,0.994517,Would Like to Hear From DP's
...,...,...,...,...,...,...
220,1325783594,0.642755,0.514265,0.613999,0.590339,West Rejects Soviet Demands On Repatriation
196,1287231546,0.688429,0.518910,0.551753,0.586364,'Give the Immigrant Work'
54,1313922463,0.506997,0.550243,0.646544,0.567928,LAKE CAMP GOAL IS 500 FROM CITY DOWNTOWN AREAS
125,1316359071,0.508396,0.573182,0.580328,0.553969,Nothing to Boast About


In [15]:
writer = open('to_export_last_round.csv', 'w')
count=0
cap=1000
idx=0
while idx<df.shape[0]:
    id_,y1,y2,y3,avg,title = df.iloc[idx,:]
#     title,text = TDMStudio.get_title_and_text(id2file(id_))
    title=title.replace('\n','')
#     if about_canada(f'{title}. {text}'):
    str_ = f'{id_}\t{y1:4.3f}\t{y2:4.3f}\t{y3:4.3f}\t{title}\n'
    writer.write(str_)
    idx+=1
writer.close()
!head to_export_last_round.csv
!echo ....
!tail to_export_last_round.csv

1287517171	1.000	1.000	1.000	ADMIT MORE DP's
1291317371	0.996	1.000	1.000	More DP's Arrive
1313779779	0.995	1.000	1.000	PARTY ARRANGED FOR 250 DP GIRLS
1291252277	1.000	0.995	1.000	78 DP Girls Reach Ontario To Take Position As Domestics
1291513884	0.987	0.996	1.000	Would Like to Hear From DP's
1313999046	0.982	1.000	1.000	Should Accept Jew DP's: Rabbi
1314019951	0.988	0.992	1.000	Wed in DP Camp, Take Vow Again
1291196990	0.979	1.000	1.000	HELP FOR DP's
1323652049	0.982	0.997	0.997	Truman in Plea For Europe's DP's
1291483011	1.000	0.994	0.980	DP's Arrive to Work in Welland Mill
....
1293565218	0.530	0.695	0.599	Soviet Wave of Terror To Hit Czechs: Nemec
1288810306	0.583	0.679	0.562	Person to Person
1291714517	0.521	0.654	0.639	The stranger
1282705175	0.593	0.557	0.642	Horror Story, Happy ending
1289153107	0.715	0.568	0.508	Home Iransplanted
1325783594	0.643	0.514	0.614	West Rejects Soviet Demands On Repatriation
1287231546	0.688	0.519	0.552	'Give the Immigrant Work'
1313922463	0.507	0.5

In [7]:
# from utils.tdmstudio import TDMStudio
# from utils.general import id2file
# import spacy
# nlp = spacy.load('en_core_web_sm', disable=['textcat','ner','parser','tagger','lemmatizer'])
# tokenizer = nlp.tokenizer
# def tokenize(text):
#     return [token.text for token in tokenizer(text)]
# canada_locs = set(open('cities.txt','r').read().splitlines() + open('provinces.txt','r').read().splitlines()+['Canada'])

# def about_canada(text):
#     tokens = tokenize(text)
#     return any([token in canada_locs for token in tokens])

# writer = open('to_export_second_round.csv', 'w')

# idx=0
# while count<cap:
#     id_,y1,y2,y3,avg = df.iloc[idx,:]
#     title,text = TDMStudio.get_title_and_text(id2file(id_))
#     title=title.replace('\n','')
#     if about_canada(f'{title}. {text}'):
# #         str_ = f'{count+1:>3}: {id_}\t{y1:4.3f}\t{y2:4.3f}\t{y3:4.3f}\t{avg:4.3f}\thttps://proquest.com/docview/{id_}\t{title}\n'
#         str_ = f'{id_}\t{y1:4.3f}\t{y2:4.3f}\t{y3:4.3f}\t{title}\n'
#         writer.write(str_)
#         count+=1
#     idx+=1
# writer.close()
# # info(f'Number of discarded for not mentioning Canada: {idx-cap}')
# !head to_export_second_round.csv

2022-03-11 13:26:01.834448 [ INFO  ] Number of discarded for not mentioning Canada: 660
1323412960	0.976	1.000	0.967	Jews in U.S. Zone Insist on Going On to Palestine
1291263794	0.948	1.000	0.992	Apparel Made by Skilful DP's Impresses Senate Committee
1287505669	0.959	0.994	0.974	Textile Workers
1287587801	0.983	0.988	0.956	Polish Writers Under 'Red' Pressure
1412095279	0.990	0.971	0.956	Welcome Mat
1325795691	0.982	0.997	0.927	OP's Influx Here Hits New Record Of 100 Each Day
1284729689	0.950	0.982	0.971	314 Skilled Migrants Signed in West Indies
1325698778	0.991	0.996	0.911	Canada Shortly Will Admit 6,000 Disked Persons
1287413237	0.929	1.000	0.964	Ottawa Slows Down Flow of Immigrants
1270515457	0.976	0.949	0.966	Immigration


In [8]:
!ls -lh to_export_second_round.csv

-rw-rw-r-- 1 ec2-user ec2-user 61K Mar 11 13:26 to_export_second_round.csv


In [30]:
#proportion
mask1 = (df['yhat1']>0.5).values
mask2 = (df['yhat2']>0.5).values
mask3 = (df['yhat3']>0.5).values

meeting_requirment = df.iloc[mask1 & mask2 & mask3,:]

info(f'{(meeting_requirment.shape[0]/df.shape[0])*100:04.2f} %')

2022-03-09 14:44:33.392658 [ INFO  ] 01.23 %
